## Goal -- how does our precision, sensitivity, and accuracy get impacted when applying the PODER filters? 

In [1]:
import pandas as pd
import numpy as np
import scipy.stats as st
import seaborn as sns
import sys
import os
import matplotlib.pyplot as plt
import yaml
from snakemake.io import expand
import pyranges as pr
from pyfaidx import Fasta
from mizani.formatters import percent_format
from scipy import stats


p = os.path.dirname(os.getcwd())
sys.path.append(p)

from scripts.utils import *
from scripts.vcf_utils import *
from scripts.plotting import *

from plotnine import *

In [2]:
def my_theme(base_size=11, w=4, h=3):
    """
    Custom plotnine theme with:
    - White background
    - Clean styling
    - Axes and ticks retained

    Parameters:
    - base_size: Base font size

    Returns:
    - plotnine.theme object
    """
    return (
        theme_minimal(base_size=base_size)
        + theme(
            # White background
            panel_background=element_rect(fill='white', color=None),
            plot_background=element_rect(fill='white', color=None),

            # Remove grid lines
            panel_grid_major=element_blank(),
            panel_grid_minor=element_blank(),
            panel_border=element_blank(),

            # Keep axis lines & ticks (don't blank them)
            axis_line=element_line(color='black'),
            axis_ticks=element_line(color='black'),

            plot_title=element_text(hjust=0.5, family='Helvetica'),
            axis_title_x=element_text(hjust=0.5, family='Helvetica'),
            axis_title_y=element_text(hjust=0.5, margin={'t':0, 'r':-2, 'b':0, 'l':0}, family='Helvetica'),
            
            # Styling text
            legend_title=element_blank(),
            axis_title=element_text(size=base_size + 1, family='Helvetica'),
            legend_text=element_text(size=base_size-2, family='Helvetica'),
            axis_text=element_text(size=base_size, color='black', family='Helvetica'),
            figure_size=(w, h),  # Controls plot dimensions (width x height in inches)
            plot_margin=0.05      # Shrinks surrounding white space
        )
    )

def clean_figure(ax):
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.tick_params(axis="x", rotation=45)

In [3]:
config = load_config()
od = '../'
meta_df = load_meta()
meta_df['lab_sample'] = meta_df['lab_number_sample'].astype(str)+'_'+\
                        meta_df['lab_sampleid']+'_'+\
                        meta_df['cell_line_id']


def proc_cfg(entry, od):
    entry = entry.replace('../../', '')
    entry = od+entry
    return entry

init_plot_settings()

In [4]:
## SQANTI reads stuff
f = proc_cfg(expand(config['lr']['qc_sirvs']['sqanti_reads']['class_summary'],
                    annot_completeness='C')[0], od)
df = pd.read_csv(f, sep='\t')

# rename structural categories
m = {'antisense': 'Antisense',
     'full-splice_match': 'FSM',
     'genic': 'Genic', 
     'incomplete-splice_match': 'ISM',
     'intergenic': 'Intergenic',
     'novel_in_catalog': 'NIC',
     'novel_not_in_catalog': 'NNC'}
df['structural_category'] = df.structural_category.map(m)

# add antisense gene ids as assc_gene_2
df['assc_gene_2'] = df.associated_gene
inds = df.loc[df.structural_category=='Antisense'].index
df.loc[inds, 'assc_gene_2'] = df.loc[inds, 'assc_gene_2'].str.split('_', expand=True)[1]

# add relevant metadata
df = df.merge(meta_df[['lab_sample', 'population']], 
              how='left', on='lab_sample')

# add # mapped reads 
df['n_mapped_reads'] = df.groupby('lab_sample')['isoform'].transform('count')

# add spike type
df['spike_type'] = np.nan
df.loc[df.chrom.str.contains('ERCC'), 'spike_type'] = 'ERCC'
df.loc[df.chrom.str.contains('SIRV'), 'spike_type'] = 'SIRV'

spliced_sirvs = ['SIRV1', 'SIRV2', 'SIRV3', 'SIRV4', 'SIRV5', 'SIRV6', 'SIRV7']
df.loc[df.chrom.isin(spliced_sirvs), 'spike_type_2'] = 'spliced_sirv'

# add # spliced sirv reads
df['n_spliced_sirv_reads'] = df.loc[df.spike_type_2=='spliced_sirv'].groupby('lab_sample')['isoform'].transform('count')

# splicing novelty
df['splicing_novelty'] = 'Novel'
df.loc[df.structural_category.isin(['FSM', 'ISM']), 'splicing_novelty'] = 'Known'

# overal known vs. novel
df['overall_nov'] = 'Novel'
df.loc[df.structural_category == 'FSM', 'overall_nov'] = 'Known'

## Ground truth sirv / ercc gtf
gtf_df = pr.read_gtf('../data/qc_sirvs/SIRV_ERCC_longSIRV_multi-fasta_20210507.gtf').df

temp = gtf_df.loc[gtf_df.Chromosome.isin(spliced_sirvs)]
n_spliced_sirvs_tot = len(temp.transcript_id.unique())

# there are 69 spliced sirvs as ground truth

# get the transcript length of each sirv transcript
gtf_df['exon_len'] = gtf_df['End'] - gtf_df['Start']
gtf_df['transcript_len'] = gtf_df.groupby('transcript_id')['exon_len'].transform('sum')

gtf_df['mean_transcript_len'] = gtf_df.groupby('gene_id')['transcript_len'].transform('mean')
gtf_df['med_transcript_len'] = gtf_df.groupby('gene_id')['transcript_len'].transform('median')

# also get number of exons
gtf_df['n_exons'] = gtf_df.groupby('transcript_id')['exon_assignment'].transform('count')
gtf_df['mean_n_exons'] = gtf_df.groupby('gene_id')['n_exons'].transform('mean')
gtf_df['med_n_exons'] = gtf_df.groupby('gene_id')['n_exons'].transform('median')

# also get number of transcripts
gtf_df['n_transcripts'] = gtf_df.groupby('gene_id')['transcript_id'].transform('nunique')

# get # monoexonic transcripts
temp = gtf_df[['gene_id', 'transcript_id', 'n_exons']].groupby(['gene_id', 'n_exons']).nunique().reset_index()
temp = temp.loc[temp.n_exons==1]
temp.drop('n_exons', axis=1, inplace=True)
temp.rename({'transcript_id': 'n_monoexonic_transcripts'}, axis=1, inplace=True)
temp.loc[temp.gene_id=='SIRV6']
gtf_df = gtf_df.merge(temp,
                      how='left',
                      on='gene_id')

# sort the long sirvs at least by length
sirv_order = ['1', '2', '3', '4', '5', '6', '7',
              '4001', '4002', '4003',
              '6001', '6002', '6003', 
              '8001', '8002', '8003',
              '10001', '10002', '10003',
              '12001', '12002', '12003']
sirv_order = [f'SIRV{s}' for s in sirv_order]

/var/folders/ch/bc1lrgbd1fn_3gxz886mxjz00000gn/T/ipykernel_85645/2818831187.py:30: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'ERCC' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.


## Apply filters one by one and create a label to say whether the transcript passed each filter 
* FSM? (pass_fsm_filter)
* Monoexonic? (pass_monoexonic_filter)
* Reproducibile for novel transcripts? (pass_reproducibility_filter)
* Promoted ISM? (promoted_ism_filter)
* ISM? (pass_ism_filter)

In [5]:
# only long sirvs
temp = df.loc[df.spike_type_2 == 'spliced_sirv']

# filter 1 -- novel transcripts must be present in 2+ samples
temp['sample_sharing'] = temp.groupby('jxnHash')['lab_sample'].transform('nunique')
temp.loc[temp.sample_sharing>=2, 'filter_pass'] = True

# filter 2 - for ISMs, if associated_transcript is not in original collection, add it and pass it

# first, mark all ISMs as non-passing
temp.loc[temp.structural_category=='ISM', 'filter_pass'] = False

# get a random fsm entry to add for each uniq one
fsm_temp = temp.loc[(temp.structural_category=='FSM')].drop_duplicates(subset='associated_transcript', keep='first')
fsm_temp['lab_sample'] = np.nan
assert len(fsm_temp.index) == n_spliced_sirvs_tot

# now loop through lab samples and tack on undet. entries FSMs
# for all ISMs
add_df = pd.DataFrame()
for s in temp.lab_sample.unique():
    undet_fsms_from_isms = temp.loc[(temp.lab_sample==s)&\
                                    (temp.structural_category=='ISM')]
    
    temp2 = temp.loc[temp.lab_sample==s]

    # these are the ids of transcripts that are detected via isms
    ism_fsm_ids = temp2.loc[(temp2.structural_category=='ISM'),
                           'associated_transcript'].unique().tolist()
    
    # these are the ids of transcripts that are detected via fsms
    fsm_ids = temp2.loc[(temp2.structural_category=='FSM'),
                        'associated_transcript'].unique().tolist()
    
    # the set difference will be the ones we need to "promote"
    promote_fsm_ids = list(set(ism_fsm_ids)-set(fsm_ids))
    promote_fsm_ids
    
    add_df_temp = fsm_temp.loc[fsm_temp.associated_transcript.isin(promote_fsm_ids)]
    add_df_temp['lab_sample'] = s
    add_df_temp['promoted_ism'] = True
    
    add_df = pd.concat([add_df, add_df_temp], axis=0)

temp = pd.concat([temp, add_df], axis=0)

# filter 0 -- all FSMs pass filtering
temp.loc[temp.structural_category == 'FSM', 'filter_pass'] = True

# filter 3 -- all monoexonics fail filtering 
temp.loc[temp.exons == 1, 'filter_pass'] = False

/var/folders/ch/bc1lrgbd1fn_3gxz886mxjz00000gn/T/ipykernel_85645/3257686515.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/var/folders/ch/bc1lrgbd1fn_3gxz886mxjz00000gn/T/ipykernel_85645/3257686515.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/var/folders/ch/bc1lrgbd1fn_3gxz886mxjz00000gn/T/ipykernel_85645/3257686515.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

In [6]:
# now reduce to just uniq junction chains
temp['ic_id'] = temp.jxn_string
temp.loc[temp.structural_category=='FSM', 'ic_id'] = temp.loc[temp.structural_category=='FSM', 'associated_transcript']

# annotate true positives
temp['tp'] = False
temp.loc[temp.structural_category=='FSM', 'tp'] = True

keep_cols = ['exons', 'assc_gene_2', 'population',
             'n_mapped_reads', 'spike_type', 'spike_type_2', 'n_spliced_sirv_reads',
             'splicing_novelty', 'overall_nov', 'sample_sharing', 'promoted_ism',
             'ic_id', 'filter_pass', 'lab_sample', 'tp']
temp = temp[keep_cols].drop_duplicates()

In [7]:
print(len(temp.index))
print(len(temp[['ic_id', 'filter_pass', 'lab_sample']].drop_duplicates()))
assert len(temp.index) == len(temp[['ic_id', 'filter_pass', 'lab_sample']].drop_duplicates())

9020
9020


In [13]:
stat_df_filt = pd.DataFrame()
stat_df_unfilt = pd.DataFrame()


# add # tp for each sample
temp['n_det_per_cat'] = temp.groupby(['lab_sample', 'tp', 'filter_pass'])['ic_id'].transform('nunique')
stat_df_filt = pd.concat([stat_df_filt, temp.loc[(temp.tp==True)&\
                                       (temp.filter_pass==True)][['lab_sample', 'n_det_per_cat']]], axis=0)
stat_df_filt.rename({'n_det_per_cat':'n_tp'}, axis=1, inplace=True)
stat_df_unfilt = pd.concat([stat_df_unfilt, temp.loc[(temp.tp==True)][['lab_sample', 'n_det_per_cat']]], axis=0)
stat_df_unfilt.rename({'n_det_per_cat':'n_tp'}, axis=1, inplace=True)


# add # fp for each sample. for novels, have to use jxnHash instead of ID
temp['n_det_per_cat_2'] = temp.groupby(['lab_sample', 'overall_nov', 'filter_pass'])['ic_id'].transform('nunique')
stat_df_filt = stat_df_filt.merge(temp.loc[(temp.overall_nov=='Novel')&\
                                 (temp.filter_pass==True)][['lab_sample', 'n_det_per_cat_2']], 
                       how='left',
                       on='lab_sample')
stat_df_filt.rename({'n_det_per_cat_2':'n_fp'}, axis=1, inplace=True)
stat_df_unfilt = stat_df_unfilt.merge(temp.loc[(temp.overall_nov=='Novel')][['lab_sample', 'n_det_per_cat_2']], 
                       how='left',
                       on='lab_sample')
stat_df_unfilt.rename({'n_det_per_cat_2':'n_fp'}, axis=1, inplace=True)


# add # fn for each sample. 
tids = gtf_df.loc[gtf_df.gene_id.isin(spliced_sirvs)].transcript_id.unique().tolist()
assert len(tids) == 69
stat_df_filt['n_fn'] = len(tids)-stat_df_filt['n_tp']
stat_df_unfilt['n_fn'] = len(tids)-stat_df_unfilt['n_tp']


# add em together after adding a filtered / unfiltered status for each 
stat_df_filt['filter_status'] = 'filtered'
stat_df_unfilt['filter_status'] = 'unfiltered'
stat_df = pd.concat([stat_df_filt, stat_df_unfilt], axis=0)
stat_df = stat_df.drop_duplicates()
assert len(stat_df.index) == 43*2 # for filtered and unfiltered

In [14]:
temp.loc[(temp.tp=='True')][['lab_sample', 'n_det_per_cat']]

,lab_sample,n_det_per_cat


In [ ]:


# stat_df['prec'] = stat_df['n_tp']/(stat_df['n_tp']+stat_df['n_fp'])
# stat_df['sens'] = stat_df['n_tp']/(stat_df['n_tp']+stat_df['n_fn'])
# stat_df['f1'] = stat_df['n_tp']/(stat_df['n_tp']+0.5*(stat_df['n_fp']+stat_df['n_fn']))

# stat_df = stat_df.merge(meta_df[['lab_sample', 'population']],
#                         how='left',
#                         on='lab_sample')

# value_vars = ['prec', 'sens', 'f1']
# stat_melted = pd.melt(
#     stat_df,
#     id_vars=[col for col in stat_df.columns if col not in value_vars],
#     value_vars=value_vars,
#     var_name='stat_type',
#     value_name='stat_val')

# stat_melted['stat_type'] = stat_melted.stat_type.map({'prec':'Precision',
#                                                       'sens':'Sensitivity',
#                                                       'f1':'F1'})